In [55]:
import pandas as pd
import json
df = pd.read_excel('agentsched.xlsx')


header_row = 6
headers = df.iloc[header_row].tolist()
headers[3] = "AgentID"
headers[4] = "AgentName"

# Shift headers from col 6 onward (remember python is 0-based)
start_col = 6
for i in range(len(headers)-1, start_col-1, -1):  # walk backwards
    headers[i] = headers[i-1]

# Now assign shifted headers and re-slice dataframe
df = df.iloc[header_row+1:].copy()   # drop the header rows above
df.columns = headers
c1 = df.columns[3]
c2 = df.columns[4]
c3 = df.columns[7]
c4 = df.columns[10]
c5 = df.columns[13]
c6 = df.columns[16]
c7 = df.columns[19]
c8 = df.columns[22]
c9 = df.columns[26]
colsToKeep = [c1,c2,c3,c4,c5,c6,c7,c8,c9]
# for col in df.columns:
#     if not col in colsToKeep:
#         df = df.drop(col, axis=1)
# Reset index so it looks neat
df = df.loc[:, df.columns.isin(colsToKeep)]
df.iloc[:, :2] = df.iloc[:, :2].shift(1).replace("\n", "", regex=True)
date_cols = [c for c in df.columns if c not in ("AgentID", "AgentName")]
keep = ~df[date_cols].isna().all(axis=1)
df = df.loc[keep].reset_index(drop=True)
# df.columns = ['AgentID','AgentName', c3, c4, c5, c6, c7, c8, c9]
df = df.reset_index(drop=True) 
df.to_json('agentsched.json', orient='records')
df = df.to_dict(orient='records')
agentsAndDetails = {}
for agentDict in df:
    aid = agentDict['AgentID']
    agentsAndDetails[aid] = agentDict
df2 = pd.DataFrame.from_dict(agentsAndDetails, orient='index')
json.dump(agentsAndDetails, open('agentsAndDetails.json', 'w'))
supejson = json.load(open('supervisors.json'))
supervisors = supejson['supervisors']
print (supervisors)
supeTeams = {}
agentNames = []
for supe in supervisors:
    supeName = supe['supervisor']
    supeTeams[supeName] = {}
    for agent in supe['agents']:
        agentNames.append(agent['agent'])
        supeTeams[supe['supervisor']][agent['agent']] = agent
        
missMatches = []
conversions = {}
for agentID in agentsAndDetails.keys():
    agn = agentsAndDetails[agentID]['AgentName'].upper()
    lf = False
    ff = False
    if agn not in agentNames:
        fn = agn.split(", ")[1].split(" ")[0].upper()
        ln = agn.split(",")[0].upper()
        if ln in [a.split(", ")[0] for a in agentNames]:
            match = input(f'{agn} possibleMatch := {[a for a in agentNames if a.split(", ")[0] == ln]}, 0 for first option, 1 for second etc, n for no match')
            if match != 'n':
                conversions[agn] = [a for a in agentNames if a.split(", ")[0] == ln][int(match)]
                lf = True
        elif fn in [a.split(", ")[1] for a in agentNames]:
            match = input(f'{agn} possibleMatch := {[a for a in agentNames if a.split(", ")[1] == fn]}, 0 for first option, 1 for second etc, n for no match')
            if match != 'n':
                conversions[agn] = [a for a in agentNames if a.split(", ")[1] == fn][int(match)]
                ff = True
        if not lf and not ff:
            missMatches.append(agn)
        lf = False
        ff = False
json.dump(supeTeams, open('supeTeams.json', 'w'))
json.dump(missMatches, open('missMatches.json', 'w'))
json.dump(conversions, open('conversions.json', 'w'))
# print(agentNames)

c:\Users\rbjorklund\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


[{'supervisor': 'ADAMS', 'overall': 0.7736159125496489, 'scores': {'AME': 0.8000000000000002, 'Completed COR': 0.8795236006798285, 'Completed COR %': 0.8795236006798285, 'Completed AAR': 0.3511857976585519, 'Completed AAR %': 0.3511857976585519, 'Completed COR AAR %': 0.734862118496608, 'Adherence %': 0.7505724766397016, 'Survey_Adw': 0.9143627149371818, 'ACD_min': 0.4712962962962963}, 'raw': {'AME': 2.0, 'Completed COR': 124.0, 'Completed COR %': 0.04343626370258516, 'Completed AAR': 54.0, 'Completed AAR %': 0.01794789091919702, 'Completed COR AAR %': 0.06138415462178215, 'Adherence %': 0.8843058724235159, 'Survey_Adw': 0.8794867790460867, 'ACD_min': 6.253703703703703}, 'agents': [{'agent': 'ARRINGTON, TIKAH', 'overall': 0.812583956969235, 'raw': {'AME': 2.0, 'Completed COR': 2.0, 'Completed COR %': 0.0227272727272727, 'Completed AAR': 3.0, 'Completed AAR %': 0.0340909090909091, 'Completed COR AAR %': 0.0568181818181818, 'Adherence %': 0.884803921568627, 'Survey_Adw': 0.9, 'ACD_min': 